<h1> 1. Import all necessary information and create functions to make parameters <h1>


<h2> a. Import necessary libraries <h2>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date

<h2> b. Acquire necassary information from deadheads and kilometers <h2>

In [2]:
information = pd.read_excel(r"/content/gdrive/My Drive/data logistics/DataCase1.xlsx", sheet_name="General")

# Make matrix of all deadhead times
# Select all rows in first sheet that contain information about deadhead times
information_deadhead = information.iloc[5:11,:]
# Reset index of rows
information_deadhead = information_deadhead.reset_index(drop=True)
# Name the index as Location so that it is clear what the index is about (in this case: Location A, Location B ect.)
information_deadhead.iloc[0,0] = "Location"
# Change the columns so that columns are also Location A, Location B, ect. such that we will have a matrix with the locations in the columns and rows and the values of deadhead in the matrix.
columns_deadhead = information_deadhead.iloc[0,:]
information_deadhead.columns = columns_deadhead
# Delete first row wich used to the names of the location (they are now set as the column names)
information_deadhead = information_deadhead.iloc[1:,:]
information_deadhead = information_deadhead.set_index('Location')

# Select all rows in first sheet that contain information about distances
kilometers_information = information.iloc[13:19,:]
# Name the index as Location so that it is clear what the index is about (in this case: Location A, Location B ect.)
kilometers_information.iloc[0,0] = "Location"
# Change the columns so that columns are also Location A, Location B, ect. such that we will have a matrix with the locations in the columns and rows and the values of kilometer in the matrix.
columns_deadhead = kilometers_information.iloc[0,:]
kilometers_information.columns = columns_deadhead
# Delete first row wich used to the names of the location (they are now set as the column names)
kilometers_information = kilometers_information.iloc[1:,:]
kilometers_information = kilometers_information.set_index('Location')

<ipython-input-2-8d275872aea1>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilometers_information.iloc[0,0] = "Location"


<h2> c. Funtion to find all feasible trips that can be added to a given duty given <h2>


In [3]:
# Give the list of feasible duties so we can make it larger when we find new feasible duties in the funciton (feasible duties)
# Give the duty we want to find all feasible from (duty)
# Give data with information of times of trips (bus_planning_df)
def find_feasible_duties(duty, feasible_duties, bus_planning_df):
  # Take the last trip in the duty, this trip has just ended we want to find if there are other feasible trips
  last_trip = duty[-1]
  # Take index of that trip, since index start at 0
  index = last_trip - 1
  # Take end time and location of the last trip made in the duty
  end_time_trip = bus_planning_df.iloc[index,3]
  end_location_trip = str(bus_planning_df.iloc[index,1])


  for j in range (0,len(bus_planning_df.index)):
    # Make copy of current duty
    new_duty = duty[:]
    # Take for all possible trips start time and location to see if compatible
    start_time_next = bus_planning_df.iloc[j,2]
    start_location_next = str(bus_planning_df.iloc[j,0])
    
    # Compute time difference in hours and minutes
    time_diff = datetime.combine(date.today(), start_time_next) - datetime.combine(date.today(), end_time_trip)
    days, seconds = time_diff.days, time_diff.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60

    # If last trip of duty compatible with selected trip j and no deadhead (i.e. same location), append this trip to new duty
    if (hours >= 0) & (hours < 1) & (minutes <= 50) & (start_location_next == end_location_trip):
          new_duty.append(j+1)
          # Append new created duty to feasible duties
          feasible_duties.append(new_duty)
  

  # When all new duties have been created and added to feasible duties return the bigger list of feasible duties
  return feasible_duties


<h2> d. Function to make covering matrix with list of feasible duties <h2>

In [4]:
def create_covering_matrix(feas_duties, bus_plan_df):
  # Initiate zeros matrix to start the covering matrix for the constraint
  cover_matrix = pd.DataFrame(np.zeros((len(bus_plan_df.index), len(feas_duties))))

  # Loop over all columns (i.e. duties)
  for i in range(0,len(feas_duties)):
    # Select a duty
    duty = feas_duties[i]
    for j in range(0, len(duty)):
        # For every trip in that duty we want to take the index and give it 1 for the assigned column of that duty
        trip = duty[j]
        index = trip - 1
        cover_matrix.iloc[index,i] = 1

  return cover_matrix


<h2> e. Function to make cost vector <h2>

In [12]:
def create_cost_vector(feas_duties, bus_plan_df):
    # Initiate vector of cost of all duties 
  cost = [0] * len(feas_duties)

  # Create cost for every duty
  for i in range(0,len(feas_duties)):
      # Select duty i
      duty = feas_duties[i]
      # Select index of first trip for duty i
      first_trip = (duty[0] - 1)
      # See what starting location is of first trip
      location_first = bus_plan_df.iloc[first_trip,0]
      location_first_str = "Location " + location_first
      # Select km and deadheads to the first start location
      kilometers = kilometers_information[location_first_str]
      deadhead = information_deadhead[location_first_str]
      # Select km and deadheads from depot to first start location
      kilometers_information_first = kilometers.iloc[0]
      deadhead_information_first = deadhead.iloc[0]

      # Add costs depending to traveltime, traveldistance and fixed costs
      cost[i] = 2*kilometers_information_first + 0.5*deadhead_information_first + 100

      # Select last trip of duty i
      last_trip = (duty[-1] - 1)
      # See what ending location of last trip is
      location_last = bus_plan_df.iloc[last_trip,1]
      location_last_str = "Location " + location_last
      # Select all km and deadhead to/from last location
      kilometers = kilometers_information[location_last_str]
      deadhead = information_deadhead[location_last_str]
      # Select km and deadheads from last location to depot
      kilometers_information_last = kilometers.iloc[0]
      deadhead_information_last = deadhead.iloc[0]

      # Add costs to depot to last location
      cost[i] =  cost[i] + (2*kilometers_information_last + 0.5*deadhead_information_last)


      # Loop over all elements in duty to find traveltime and costs in between
      for j in range(0,len(duty)-1):
        # Take first and and second elements of duty
        index_trip = duty[j] - 1
        index_next = duty[j+1] - 1

        # Select their end trip and next start trip
        end_time_trip = bus_plan_df.iloc[index_trip,3]
        start_time_next = bus_plan_df.iloc[index_next,2]

        # Compute time difference 
        time_diff = datetime.combine(date.today(), start_time_next) - datetime.combine(date.today(), end_time_trip)
        days, seconds = time_diff.days, time_diff.seconds
        hours = days * 24 + seconds // 3600
        minutes = (seconds % 3600) // 60

        # Add waiting time between trips in duty to costs
        cost[i] =  cost[i] + minutes * 0.5 + hours * 30


  cost_vector = pd.DataFrame(cost, columns=['costs'])

  return cost_vector

<h1> 2. Monday-Friday set covering formulation <h1>
 <h2> a. Find all feasible duties Monday-Friday, Saturday and Sunday <h2>

In [6]:
# Import informaiton of bus trips for monda-friday
mofri_df = pd.read_excel(r"/content/gdrive/My Drive/data logistics/DataCase1.xlsx", sheet_name="Monday-Friday")
 
# Initiate list of all feasible duties
feasible_duties_mf = []

# Make a column to number independent the trips in the data
mofri_df['Trip'] = list(range(1,(len(mofri_df.index)+1)))

# Initiate feasible duties by starting with all single trips
# TD - (loop good??)
for i in range(0, len(mofri_df.index)):
  trip = mofri_df.iloc[i,4]
  feasible_duties_mf.append([trip])

# For every duty in feasible duties add all possible next trips that are feasible as a new duty through the function
for duty in feasible_duties_mf:
    feasible_duties_mf = find_feasible_duties(duty, feasible_duties_mf, mofri_df)



In [7]:
# Import informaiton of bus trips for monda-friday
Saturday_df = pd.read_excel(r"/content/gdrive/My Drive/data logistics/DataCase1.xlsx", sheet_name="Saturday")
 
# Initiate list of all feasible duties
feasible_duties_sat = []

# Make a column to number independent the trips in the data
Saturday_df['Trip'] = list(range(1,(len(Saturday_df.index)+1)))

# Initiate feasible duties by starting with all single trips
# TD - (loop good??)
for i in range(0, len(Saturday_df.index)):
  trip = Saturday_df.iloc[i,4]
  feasible_duties_sat.append([trip])

# For every duty in feasible duties add all possible next trips that are feasible as a new duty through the function
for duty in feasible_duties_sat:
    feasible_duties_sat = find_feasible_duties(duty, feasible_duties_sat, Saturday_df)


In [9]:
# Import informaiton of bus trips for monda-friday
sunday_df = pd.read_excel(r"/content/gdrive/My Drive/data logistics/DataCase1.xlsx", sheet_name="Sunday")
 
# Initiate list of all feasible duties
feasible_duties_sun = []

# Make a column to number independent the trips in the data
sunday_df['Trip'] = list(range(1,(len(sunday_df.index)+1)))

# Initiate feasible duties by starting with all single trips
# TD - (loop good??)
for i in range(0, len(sunday_df.index)):
  trip = sunday_df.iloc[i,4]
  feasible_duties_sun.append([trip])

# For every duty in feasible duties add all possible next trips that are feasible as a new duty through the function
for duty in feasible_duties_sun:
    feasible_duties_sun = find_feasible_duties(duty, feasible_duties_sun, sunday_df)


<h2> b. Create covering matrix for Monday-Friday, Saturday and Sunday <h2>

In [10]:
covering_mofri = create_covering_matrix(feasible_duties_mf, mofri_df)
covering_sat = create_covering_matrix(feasible_duties_sat, Saturday_df)
covering_sun = create_covering_matrix(feasible_duties_sun, sunday_df)


covering_mofri.to_excel("/content/gdrive/My Drive/data logistics/covering_matrix_MoFri_v2.xlsx")
covering_sat.to_excel("/content/gdrive/My Drive/data logistics/covering_matrix_Sat_v2.xlsx")
covering_sun.to_csv("/content/gdrive/My Drive/data logistics/covering_matrix_Sun_v2.csv")



<h2> c. Create cost vectors for Monday-Friday, Saturday and Sunday <h2>

In [11]:
cost_mofri = create_cost_vector(feasible_duties_mf, mofri_df)
cost_sat = create_cost_vector(feasible_duties_sat, Saturday_df)
cost_sun = create_cost_vector(feasible_duties_sun, sunday_df)

cost_mofri.to_excel("/content/gdrive/My Drive/data logistics/Cost_vector_MoFri_v2.xlsx")
cost_sat.to_excel("/content/gdrive/My Drive/data logistics/Cost_vector_Saturday_v2.xlsx")
cost_sun.to_excel("/content/gdrive/My Drive/data logistics/Cost_vector_Sunday_v2.xlsx")


[105, 29, 109, 33, 113, 37, 117]
[79, 29, 83, 33, 87, 37, 91]
[118, 54, 147, 45, 138, 36, 129, 182]
